In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from pathlib import Path
from sklearn.model_selection import train_test_split

import tensorflow as tf

from sklearn.metrics import confusion_matrix, classification_report



In [2]:
negative_dir = Path('/content/drive/MyDrive/non-stained')
positive_dir = Path('/content/drive/MyDrive/stained')

In [3]:
def generate_df(image_dir, label):
    filepaths = pd.Series(list(image_dir.glob(r'*.jpg')), name='Filepath').astype(str)
    labels = pd.Series(label, name='Label', index=filepaths.index)
    df = pd.concat([filepaths, labels], axis=1)
    return df

In [5]:
positive_df = generate_df(positive_dir, label="POSITIVE")
negative_df = generate_df(negative_dir, label="NEGATIVE")

all_df = pd.concat([positive_df, negative_df], axis=0).sample(frac=1.0, random_state=1).reset_index(drop=True)
all_df

,Filepath,Label
0,/content/drive/MyDrive/stained/Copy of Image_4...,POSITIVE
1,/content/drive/MyDrive/stained/Copy of Image_8...,POSITIVE
2,/content/drive/MyDrive/stained/Copy of Image_1...,POSITIVE
3,/content/drive/MyDrive/stained/Copy of Image_3...,POSITIVE
4,/content/drive/MyDrive/non-stained/Copy of ns1...,NEGATIVE
5,/content/drive/MyDrive/stained/Copy of Image_3...,POSITIVE
6,/content/drive/MyDrive/stained/Copy of Image_1...,POSITIVE
7,/content/drive/MyDrive/stained/Copy of Image_1...,POSITIVE
8,/content/drive/MyDrive/stained/Copy of Image_3...,POSITIVE
9,/content/drive/MyDrive/stained/Copy of s2.jpg,POSITIVE


In [11]:

train_df, test_df = train_test_split(
    all_df.sample(25, random_state=1),
    train_size=0.7,
    shuffle=True,
    random_state=1
)




In [12]:
train_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

test_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [18]:
train_data = train_gen.flow_from_dataframe(
    train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_data = train_gen.flow_from_dataframe(
    train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)



Found 14 validated image filenames belonging to 2 classes.
Found 3 validated image filenames belonging to 2 classes.


In [19]:
inputs = tf.keras.Input(shape=(120, 120, 3))
x = tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu')(inputs)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2, 2))(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 120, 120, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 118, 118, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 59, 59, 16)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 57, 57, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 28, 28, 32)        0         
 g2D)                                                            
                                                                 
 global_average_pooling2d (  (None, 32)                0     

In [20]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

Epoch 1/10
1/1 [==============================] - 3s 3s/step - loss: 0.7616 - accuracy: 0.2857 - val_loss: 0.7780 - val_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 1s 652ms/step - loss: 0.7300 - accuracy: 0.2857 - val_loss: 0.7164 - val_accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 1s 641ms/step - loss: 0.7009 - accuracy: 0.2857 - val_loss: 0.6608 - val_accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 1s 687ms/step - loss: 0.6762 - accuracy: 0.7143 - val_loss: 0.6115 - val_accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 1s 615ms/step - loss: 0.6568 - accuracy: 0.7143 - val_loss: 0.5657 - val_accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 1s 683ms/step - loss: 0.6405 - accuracy: 0.7143 - val_loss: 0.5218 - val_accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 1s 899ms/step - loss: 0.6263 - accuracy: 0.7143 - val_loss: 0.4799 - val_accuracy: 1.0000
Epoch 8/10
1/1 

In [21]:
def evaluate_model(model, test_data):

    results = model.evaluate(test_data, verbose=0)
    loss = results[0]
    acc = results[1]

    print("Test Loss: {:.5f}".format(loss))
    print("Test Accuracy: {:.2f}%".format(acc * 100))

    y_pred = np.squeeze((model.predict(test_data) >= 0.5).astype(int))
    cm = confusion_matrix(test_data.labels, y_pred)
    clr = classification_report(test_data.labels, y_pred, target_names=["NEGATIVE", "POSITIVE"])

    plt.figure(figsize=(6, 6))
    sns.heatmap(cm, annot=True, fmt='g', vmin=0, cmap='Blues', cbar=False)
    plt.xticks(ticks=np.arange(2) + 0.5, labels=["NEGATIVE", "POSITIVE"])
    plt.yticks(ticks=np.arange(2) + 0.5, labels=["NEGATIVE", "POSITIVE"])
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title("Confusion Matrix")
    plt.show()

    print("Classification Report:\n----------------------\n", clr)

In [25]:
import numpy as np
import tensorflow as tf
from google.colab import files
from tensorflow.keras.preprocessing import image

In [31]:
uploaded = files.upload()

for fn in uploaded.keys():
    # Load and preprocess the image
    img = image.load_img(fn, target_size=(120, 120))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    # Predict using the model
    prediction = model.predict(img_array)
    if prediction[0][0]<0.71:
      print("unstained")
    else:
      print("stained")

    print(prediction)




Saving clean2.jpg to clean2.jpg
1/1 [==============================] - 0s 24ms/step
unstained
[[0.70946497]]


In [32]:
from tensorflow.keras.models import load_model
model.save('/content/staindetection.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:


from google.colab import drive
drive.mount('/content/drive')